## Checklist of the project:

file opened;

file examined;

missing values identified;

missing values filled;

explanation of which missing values were found;

possible causes of gaps in data described;

explained the principle by which gaps were filled;

real data type replaced with integer;

explanation of which method is used to change the data type and why;

duplicates removed;

explanation of which method is used to find and remove duplicates;

possible causes of duplicates in data described;

lemmas in the values of the column with the purposes of obtaining a loan highlighted;

lemmatization process described;

data categorized;

explanation of the principle of data categorization;

answer to the question "Is there a relationship between having children and repaying a loan on time?";

answer to the question "Is there a relationship between marital status and repaying a loan on time?";

there is an answer to the question "Is there a relationship between the level of income and the timely repayment of the loan?";

there is an answer to the question "How do different loan purposes affect its timely repayment?";

there are conclusions at each stage;

there is a general conclusion.

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from nltk.stem import SnowballStemmer
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Data loading and EDA

In [63]:
data = pd.read_csv('/Users/amastikbayev/Desktop/credit_scoring/data.csv')

In [64]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [65]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [66]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [67]:
# lets check if are these missing values in days_employed same rows missing in total_income
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [68]:
data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]['income_type'].value_counts()

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64

In [69]:
pos_days_employed = data[data['days_employed'] > 0].shape[0]

print(f"number of people with positive days_employed: {pos_days_employed}")

neg_days_employed = data[data['days_employed'] < 0].shape[0]

print(f"number of people with negative days_employed: {neg_days_employed}")

print('number of people with positive days employed and retired',pos_days_employed & data[data['income_type'] == 'пенсионер'].shape[0])

print('number of people with negative days employed and retired',neg_days_employed & data[data['income_type'] == 'пенсионер'].shape[0])

number of people with positive days_employed: 3445
number of people with negative days_employed: 15906
number of people with positive days employed and retired 3344
number of people with negative days employed and retired 3584


In [70]:
print('number of people with dob year being zero:', data[data['dob_years'] ==0 ].shape[0])
print('number of rows with negative children count:', data[data['children'] < 0].shape[0])
print('number of rows with number of children being abnoramlly high:', data[data['children'] > 10].shape[0])


number of people with dob year being zero: 101
number of rows with negative children count: 47
number of rows with number of children being abnoramlly high: 76


I got that:

~75% of the values from 'days_employed' are negative and do not apply to retirees -> I will simply take the entire column by module to remove this misunderstanding;
almost all the remaining values > 0 (except for two) are pensioners, and for all of them the length of service > at least a somewhat reasonable 90 years, which, with all due respect, looks absurd;
zero age for some clients - I assume that the data was simply not entered into the database (I exclude their absence, since this is still a loan issuance procedure). It needs to be corrected;
"negative count for children" - to be corrected;
">10 children" is not a unique case, but simply an error (maybe the correct value is "1" or "2"). Also to be corrected.

# Data Preprocessing

## 1. Handling Missing Values (and Anomalous Values)
1. Pairwise NaN values in 'days_employed' and 'total_income' will be filled with group values by 'type' relative to age.
2. Process negative values in 'days_employed' (check if absolute values can be used).
3. The 'education' column has inconsistent values (e.g., 'Среднее'-'среднее'-'СРЕДНЕЕ') - needs standardization.
4. Correct the minimum value in 'children' (-1) to 0.
5. Handle zero age values in 'dob_years' rows.
6. For pensioners > 90 years old, adjust 'days_employed' to reasonable values.
7. For 'children' > 20 - adjust to reasonable values.

## 2. Data Type Conversion
1. Convert 'days_employed' and 'total_income' columns to integer type (current precision is unnecessary).

## 3. Handling Duplicates
1. Identify duplicates, review them, and process accordingly.

## 4. Lemmatization
1. For the 'purpose' column containing similar entries with different phrasing:
   - First find unique words in their base forms (using pymystem3 module).
   - Then select a word set (excluding verbs, spaces, and prepositions) and extract their stems using nltk.stem.

## 5. Data Categorization
1. Based on lemmatization results, create a new column with numerical categories for loan purposes.
2. Verify uniqueness and correct categorization in these columns:
   - 'education' - 'education_id'
   - 'family_status' - 'family_status_id'
   (They appear correct but need validation)
3. Categorize 'days_employed' (after pensioner adjustment) into 3 groups:
   - Less than 10 years
   - 10-30 years
   - More than 30 years
4. Categorize borrowers by 'children' count:
   - No children
   - 1-2 children
   - Large families
5. Age categories ('dob_years'):
   - Under 30 years
   - 30-45 years
   - 45-65 years
   - Over 65 years

## 2.1. Handling gaps

2.1. 

1. Let's make a pivot_table for each of the 'income_type' in order to deal with negative work experience and find out who creates a bias in the average work experience value, which is equal to 1000 years.

For each 'income_type' we will output the following:

the total number of rows of this income_type in the table;
the average value for 'days_employed';
to check the correctness of calculating the average and applying abs(), we will output the number of values for 'days_employed' > 0 for this 'type'.

In [71]:
data_grouped = data.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['total number of rows', 'mean', 'number of values > 0']))

data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                                    
                total number of rows           mean number of values > 0
income_type                                                             
безработный                        2  366413.652744                    2
в декрете                          1   -3296.759962                    0
госслужащий                     1312   -3399.896902                    0
компаньон                       4577   -2111.524398                    0
пенсионер                       3443  365003.491245                 3443
предприниматель                    1    -520.848083                    0
сотрудник                      10014   -2326.499216                    0
студент                            1    -578.751554                    0

What I have:

the values for days worked ('days_employed') are of the same type: for all "unemployed" and "retired" all values are positive, for the rest are negative, so we can painlessly apply abs(), calculate the average.

### 2.1.2. I do the following

I take the values for 'days_employed' and 'salary' by module.

I add the column 'ratio_days_employed' to calculate the coefficient of completeness of the number of days worked (considering that officially you can work from the age of 16), in order to subsequently fill in NaN for 'days_employed' taking into account the average of this indicator for the group

(to avoid columns where in the column 'days_employed' NaN via notnull())

In [72]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['ratio_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


### 2.1.3. Filling in NaN gaps in the data frame data:
-'days_employed': NaN is replaced by the product of the average coefficient for the 'income_type' group multiplied by the age in days;
-'total_income': NaN is replaced by the average salary value for the 'income_type' group.

In [73]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


2.1.4. i remove the ratio days employed column and display general information on the dataframe.

In [74]:
data = data.drop('ratio_days_employed', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 2.2. Handling strange values (items 3-6)
To work on:
-'days_employed' for pensioners > 90 years old - something needs to be done. 

I think we should do the following: let the values for length of service ('days_employed') for pensioners be incorrect, 
but still, objectively, their length of service will be on average longer than that of other groups. 

Therefore, we will simply assign a length of service attribute to each user. 

And all pensioners with 60 years of service and 6000 years of service will be assigned to the same category.

-the 'education' column has different values (e.g. 'Average'-'average'-'AVERAGE') - we will need to bring them to a common form;

-correct the minimum value for 'children' (-1) to 0;

-handle zero age for some 'dob_years' rows.

In [75]:
# something that annoys immediately is education colums being in mixed case
data['education'] = data['education'].str.lower()

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [76]:
# 'children' (-1) на 0
print("number of rows with children -1 :", data[data['children'] == -1].count()[0])
print("number of rows with children 0 :", data[data['children'] == 0].count()[0])
print("number of rows with children 20 :", data[data['children'] == 20].count()[0])

number of rows with children -1 : 47
number of rows with children 0 : 14149
number of rows with children 20 : 76


In [77]:
data['children'] = data['children'].replace(-1, 0)

There are 47 lines with a value for the number of children equal to "-1", we will change it to "0". 

It will not spoil the statistics, but will look humane. Those who have 20 children are subsequently classified as having many children (>2 children).

In [78]:
print("# of rows with 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])
print("# of rows with 'dob_years' < 19 -", data[data['dob_years'] < 19].count()[0])

# of rows with 'dob_years' 0 - 101
# of rows with 'dob_years' < 19 - 101


In [79]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))

The number of some and others is the same => these are the same people. Let's fill in the values (rounding to keep 'int' for this column) for them with the average for each group

In [80]:
print("# of rows with 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])

# of rows with 'dob_years' 0 - 0


We convert the monthly salary and length of service in days to 'int'. We use the 'astype' function, since our columns have a pure 'float64' format, i.e. converting to 'int' will be safe.

In [81]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [82]:
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

Все данные представлены в довольно удобном формате. Месячную зарплату и стаж в днях перевели в 'int', чтобы видеть целочисленные значения. В остальном типа данных везде выглядят хорошо.

### 2.3. Processing duplicates

Let's count the number of duplicates again.
Using the standard "duplicated()" method, we output the total number of duplicates for the data frame:

In [83]:
data.duplicated().sum()

71

In [84]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed']).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
16204,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
1681,0,7593897635,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
4529,0,7593897635,57,среднее,1,гражданский брак,1,M,пенсионер,0,137127,свадьба
7808,0,7593897635,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7995,0,7593897635,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба


And they really do look like duplicates. The reason? I assume that it is a banal duplication of data/erroneous creation of duplicates.

I delete them.

In [85]:
data = data.drop_duplicates()

In [86]:
data.duplicated().sum()

0

Duplicates dropped

### 2.4. Lemmatization

The dataframe has a column ('purpose') that tells about the purposes of obtaining a loan. It contains similar purposes, but expressed in different words. 

It is necessary to find out the number of unique lemmas and add another column ('purpose_cat') with the category of the loan purpose.

#### 2.4.1. First, let's output a list of unique purposes ('purpose')

In [89]:
unique_purposes = data['purpose'].value_counts().index.tolist()
print(data['purpose'].value_counts().to_frame())

                                        count
purpose                                      
свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости        

In [92]:
!pip install pymystem3

In [93]:
from pymystem3 import Mystem

2.4.2. Let's use the pymystem3 module and lemmatize the list obtained above, obtaining a "rating" of unique lemmas:

In [94]:
m = Mystem()
list_of_lemmas = []
for element in data['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

Installing mystem to /Users/amastikbayev/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

#### 2.4.3. 
I think it is necessary to take the top 8 without spaces and take words 4 or more letters long - this way we will leave only nouns (they are the most popular) and will get rid of spaces and prepositions.

Let's create a separate list final_list_of_purposes with our top 8 goals for getting a loan.

In [95]:
final_list_of_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(final_list_of_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']


Let's clean up the list:
- remove "покупка", as it is too general a concept;
- remove "жилье", as it relates to недвижимость (judging by the unloading above in 2.4.1);
- remove "операция", as it also relates to housing (judging by the unloading above in 2.4.1).

In [96]:
final_list_of_purposes.remove('покупка') 
final_list_of_purposes.remove('строительство')
final_list_of_purposes.remove('операция')
print(final_list_of_purposes)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']


Let's transform the sheet into a dictionary to assign categories to the lines:

1 - real estate acquisition;

2 - car acquisition;

3 - education;

4 - wedding.

In [100]:
russian_stemmer = SnowballStemmer('russian')

stemmed_purposes = [russian_stemmer.stem(word) for word in final_list_of_purposes]
print(stemmed_purposes)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


In [101]:
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes['недвижим'] = 1
print(dict_stemmed_purposes)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


We got a list of loan purposes as a dictionary {purpose:type number}.

In the next step, among other things, we will create a new column and give each row a category from this list and check whether all the rows were "categorized".

### 2.5. Data categorization

2.5.1. Let's start by creating a new column 'purpose_cat' with categories by loan purposes 'purpose' based on the dictionary dict_stemmed_purposes we created above:

In [102]:
data['purpose_cat'] = 0
for row in range(len(data)):
    for purpose in dict_stemmed_purposes:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = dict_stemmed_purposes[purpose]
        
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


In [105]:
#checking if any null values left in purpose_cat
data[data['purpose_cat'] == 0].count().sum()

0

It means that all purposes have been assigned

In [106]:
data.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


it seems very logical that most people have high school degrees, around 20% have the bachelor's, and other degrees are logically distributed

In [107]:
data.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2810
1,в разводе,3,1195
2,вдовец / вдова,2,959
3,гражданский брак,1,4151
4,женат / замужем,0,12339


As well as family status seem very accurate and no visible illogical distribution

2.5.4. Let's split into classes by 'total_income'

-'Poor' (<= 50,000);
-'Average' (50,000 < x <= 120,000);
-'Wealthy' (120,000 < x < 1,000,000);
-'Millionaire' (x >= 1,000,000);

by length of service ('days_employed'):

-'up to 10 years';
-'10-30 years';
-'over 30 years';

by age ('dob_years'):

-'up to 30 years';
-'30-45 years';
-'45-65 years';
-'over 65 years';

by number of children ('children'):

-'no children' (0 children);
-'1-2 children';
-'large families'(>= 3 children);

For convenience, we will write functions for these purposes:

In [110]:
def salary_cat(row):
    if row['total_income'] <= 50000:
        return 'бедный'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 1000000:
        return 'зажиточный'
    else:
        return 'миллионер'

    
def days_employed_cat(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

    
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [ ]:
data['salary_cat'] = data.apply(salary_cat, axis=1)
data['days_employed_cat'] = data.apply(days_employed_cat, axis=1)
data['dob_years_cat'] = data.apply(dob_years_cat, axis=1)
data['children_cat'] = data.apply(children_cat, axis=1)

In [119]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,...,purpose,purpose_cat,income_class,experience_class,age_class,family_class,salary_cat,days_employed_cat,dob_years_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,...,покупка жилья,1,зажиточный,стаж более 30 лет,30-45 лет,1-2 ребенка,зажиточный,стаж более 30 лет,30-45 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,...,приобретение автомобиля,2,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,...,покупка жилья,1,зажиточный,стаж 10-30 лет,30-45 лет,нет детей,зажиточный,стаж 10-30 лет,30-45 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,...,дополнительное образование,3,зажиточный,стаж 10-30 лет,30-45 лет,многодетные,зажиточный,стаж 10-30 лет,30-45 лет,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,...,сыграть свадьбу,4,зажиточный,стаж более 30 лет,45-65 лет,нет детей,зажиточный,стаж более 30 лет,45-65 лет,нет детей


#### Conclusion

We categorized the data frame by the following data:

-'education' and 'family_status_id' (there was already a categorization by them initially);
-by salary level ('total_level');
-by length of service ('days_employed');
-by number of children ('children').

Let's move on to analyzing the obtained data.

#### Step 3. Answering to most important questions after setup

In [122]:
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

1. Is there a relationship between having children and repaying a loan on time?

In [132]:
pivot_table_children = data.pivot_table(index='children_cat', values='debt', aggfunc=['mean', 'count'])
pivot_table_children.columns = ['mean_debt', 'count']
pivot_table_children = pivot_table_children.sort_values(by='mean_debt')
pivot_table_children

,mean_debt,count
children_cat,,
нет детей,0.075258,14138
многодетные,0.085526,456
1-2 ребенка,0.093003,6860


Conclusion: Borrowers without children are less likely to default on loan payments.

2. Is there a relationship between age and debt?

In [133]:
pivot_table_age = data.pivot_table(index='dob_years_cat', values='debt', aggfunc=['mean', 'count'])
pivot_table_age.columns = ['mean_debt', 'count']
pivot_table_age = pivot_table_age.sort_values(by='mean_debt')
pivot_table_age

,mean_debt,count
dob_years_cat,,
старше 65 лет,0.054749,895
45-65 лет,0.063847,8771
30-45 лет,0.090962,8608
до 30 лет,0.109748,3180


Conclusion: Younger borrowers are safer 

3. Is there a relationship between marital status and repaying a loan on time?

In [134]:
pivot_table_fam = data.pivot_table(index='family_status', values='debt', aggfunc=['mean', 'count'])
pivot_table_fam.columns = ['mean_debt', 'count']
pivot_table_fam = pivot_table_fam.sort_values(by='mean_debt')
pivot_table_fam

,mean_debt,count
family_status,,
вдовец / вдова,0.065693,959
в разводе,0.071130,1195
женат / замужем,0.075452,12339
гражданский брак,0.093471,4151
Не женат / не замужем,0.097509,2810


Conclusion: Citizens who are not in a legal relationship are more likely to be late with their loan payments.

4. Is there a relationship between income level and timely loan repayment?

In [135]:
pivot_table_sal = data.pivot_table(index='salary_cat', values='debt', aggfunc=['mean', 'count'])
pivot_table_sal.columns = ['mean_debt', 'count']
pivot_table_sal = pivot_table_sal.sort_values(by='mean_debt')
pivot_table_sal

,mean_debt,count
salary_cat,,
бедный,0.061828,372
миллионер,0.080000,25
зажиточный,0.081459,14584
средний,0.081570,6473


Conclusion: Oddly enough, borrowers with an income level of < 50,000 are less likely to default on loan repayments on time.

5. How do different loan purposes affect whether you repay the loan on time?

In [136]:
pivot_table_purpose = data.pivot_table(index='purpose_cat', values='debt', aggfunc=['mean', 'count'])
pivot_table_purpose.columns = ['mean_debt', 'count']
pivot_table_purpose = pivot_table_purpose.sort_values(by='mean_debt')
pivot_table_purpose

,mean_debt,count
purpose_cat,,
1,0.072334,10811
4,0.080034,2324
3,0.092200,4013
2,0.093590,4306


Conclusion: {'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}

loans for real estate are more dangerous than education and wedding while car loan is safest

### Step 4. General conclusion

In answering the question "We need to figure out whether the client's marital status and number of children affect the fact of repaying the loan on time", 

I can give the following answer - marital status and number of children affect the fact of repaying the loan in the following lines:

- borrowers with officially registered relationships (or who were officially married in the past) and do not have children - the most responsible borrowers;

- borrowers who are in an unofficial marriage or are single, but have 1 or 2 children - the least responsible borrowers.

In general, based on the researched data, we can draw the following conclusion:

| Responsible Borrower         | Has no children        | Is/was in an official marriage           | Income level < 50,000  | Takes a loan for housing                |
|------------------------------|------------------------|-------------------------------------------|------------------------|------------------------------------------|
| Less Responsible Borrower    | Has 1–2 children       | Not in a legal relationship / not in a relationship | Income level > 50,000  | Takes a loan for education/wedding      |
